# Loading import library

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles


from scripts import metrics
import importlib as imp
imp.reload(metrics)

<module 'scripts.metrics' from '/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py'>

# Funktion for calculate metrics

In [2]:
def calculate_metrics(output_set, recall_set, number, scaffold_type, type_cluster, generator, receptor, save=True):

    mt = metrics.Metrics(type_cluster, scaffold_type, generator, number, receptor, save)     
    mt.load(output_set, recall_set)
    result = mt.calculate_metrics_with_return()

    return result

## Count metrics 
- Need the change parameters depend on which type of scaffold/cluster/generator/target you have

In [3]:
type_cluster = 'sim' #options: 'dis'|'sim' 
numbers = [0,1,2,3,4] #options: 0-4 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generator = 'Molpher' #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'


In [4]:
for number in numbers:
    a0 = calculate_metrics(f"data/output_sets/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                    f"data/input_recall_sets/{receptor}/RS_{receptor}_{type_cluster}_{number}.csv",\
                       number, scaffold_type, type_cluster,generator,receptor,True)
    a0

data/output_sets/Glucocorticoid_receptor/cOS_Molpher_sim_0_one_column.csv
Convert_
Convert_
add_columns_same_like_input_function


/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Save
data/output_sets/Glucocorticoid_receptor/cOS_Molpher_sim_1_one_column.csv
Convert_
Convert_


[21:05:37] Explicit valence for atom # 16 C, 5, is greater than permitted
[21:05:37] Explicit valence for atom # 17 C, 5, is greater than permitted


Faild to create scaffold_csk
Index 46619
CC1=CC=C(F)C=C1CC1=C(N[SH](C)(C)(O)O)C=C2NC3=CC=NC=C3C2=C1

Cc1ccc(F)cc1Cc1cc2c(cc1N[SH](C)(C)(O)O)[nH]c1ccncc12
Faild to create scaffold_csk
Index 46620
COC1=CC=C(F)C=C1CC1=C(N[SH](C)(C)(O)O)C=C2NC3=CC=NC=C3C2=C1

COc1ccc(F)cc1Cc1cc2c(cc1N[SH](C)(C)(O)O)[nH]c1ccncc12


...

Faild to create scaffold_csk
Index 881995
C=CCOC1CC1C1CCC1(C)[SH]1(C)(O)CN1C(C)C(=O)NC1=CC=CN=C1C

C=CCOC1CC1C1CCC1(C)[SH]1(C)(O)CN1C(C)C(=O)Nc1cccnc1C


[22:06:01] Explicit valence for atom # 17 C, 5, is greater than permitted
[22:06:01] Explicit valence for atom # 18 C, 5, is greater than permitted
[22:06:01] Explicit valence for atom # 19 C, 5, is greater than permitted
[22:06:01] Explicit valence for atom # 19 C, 5, is greater than permitted


Faild to create scaffold_csk
Index 1076832
CCC1(COC2=CC=C3CCC(C)CC3=C2)C[SH](C)(O)(CC2=CC=CN=C2C)N1

CCC1(COc2ccc3c(c2)CC(C)CC3)C[SH](C)(O)(Cc2cccnc2C)N1
Faild to create scaffold_csk
Index 1076833
CCC1(CCOC2=CC=C3CCC(C)CC3=C2)C[SH](C)(O)(CC2=CC=CN=C2C)N1

CCC1(CCOc2ccc3c(c2)CC(C)CC3)C[SH](C)(O)(Cc2cccnc2C)N1
Faild to create scaffold_csk
Index 1076834
CCC1(CCOC2=CC=C3CCC(C)CCC3=C2)C[SH](C)(O)(CC2=CC=CN=C2C)N1

CCC1(CCOc2ccc3c(c2)CCC(C)CC3)C[SH](C)(O)(Cc2cccnc2C)N1
Faild to create scaffold_csk
Index 1076835
CCC1(CCOC2=CC=C3CCC(C)CCC3=C2)C[SH](C)(O)(OCC2=CC=CN=C2C)N1

CCC1(CCOc2ccc3c(c2)CCC(C)CC3)C[SH](C)(O)(OCc2cccnc2C)N1
add_columns_same_like_input_function


/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/PhD/phd_projects/IGA_2023_try_metrics_for_other_generators_v2/scripts/metrics.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

Save


## Calculate acarage value and save csv
- Do this step after calculate all metrics for each cluster

In [18]:
def avarage_value(number, type_cluster, scaffold_type, generator, receptor):

    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/"
    link = {}
    for x in range(number):
        link[f'link_{x}'] = f'{path}metrics_cluster_{x}_{type_cluster}_{generator}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
        
    display(df)
    a = df.mean(numeric_only=True)
    df.loc[len(df)] = [f"{generator}_mean", type_cluster, scaffold_type, a['uniq_scaffolds'], a['set_size'], '-',\
                       a['TUPOR'], a['SESY'], a['ASER'], a['tRS']]
    df = df.round(7)
    dff = df.copy()
    dff['set_size'] = dff['set_size'].apply(lambda x : "{:,}".format(x))
    dff['uniq_scaffolds'] = dff['uniq_scaffolds'].apply(lambda x : "{:,}".format(x))
    dff['tRS'] = dff['tRS'].apply(lambda x : "{:,}".format(x))

    mean = pd.DataFrame()
    mean = df.loc[5].to_frame().T
                                        
    display(mean)
    dff.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean_with_coma.csv", index = False)
    df.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.csv", index = False)
    dff.to_html(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.html", index = False)
    mean.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False) #float_format='%.7f'

In [29]:
type_cluster = 'dis' #options: 'dis'|'sim' 
number = 5 #options: 0-4 
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generator = 'DrugEx' #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [30]:
avarage_value(number, type_cluster, scaffold_type, generator, receptor)

,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
0,DrugEx_0,dis,murcko,693078,1000000,14/101,0.138614,0.693078,0.003156,3156
1,DrugEx_1,dis,murcko,680537,1000000,4/49,0.081633,0.680537,0.000029,29
2,DrugEx_2,dis,murcko,780801,1000000,8/67,0.119403,0.780801,0.000216,216
3,DrugEx_3,dis,murcko,704301,1000000,14/83,0.168675,0.704301,0.000588,588
4,DrugEx_4,dis,murcko,685372,1000000,33/108,0.305556,0.685372,0.002277,2277


,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
5,DrugEx_mean,dis,murcko,708817.8,1000000.0,-,0.162776,0.708818,0.001253,1253.2


# Connect all mean value to one pandas DataFrame

In [18]:
def connect_mean_value(reeptor, scaffold_type, type_cluster, generators_name_list):
    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/"
    link = {}
    for generator in generators_name_list:
        link[f'link_{generator}'] = f'{path}{generator}_mean_{scaffold_type}_{type_cluster}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
    display(df)
    dff = df.copy()
    dff['set_size'] = dff['set_size'].apply(lambda x : "{:,}".format(x))
    dff['uniq_scaffolds'] = dff['uniq_scaffolds'].apply(lambda x : "{:,}".format(x))
    dff['tRS'] = dff['tRS'].apply(lambda x : "{:,}".format(x))
    
    dff.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}_with_coma.csv", index = False)
    df.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}.csv", index = False)
    dff.to_html(f"{path}/mean_{scaffold_type}_{type_cluster}.html", index = False)

In [25]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generators_name_list = ['Molpher', 'DrugEx'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [26]:
connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,uniq_scaffolds,set_size,TUPOR_,TUPOR,SESY,ASER,tRS
0,Molpher_mean,dis,murcko,264723.0,1035441.0,-,0.166131,0.258484,0.001580,1449.6
1,DrugEx_mean,dis,murcko,708817.8,1000000.0,-,0.162776,0.708818,0.001253,1253.2
